# Homework 2
Tomasz Siłkowski

## Task 1

Since we are given only the confusion matrix for group Blue and information that group Red is selected randomly, all statistics will be derived from confusion matrices, which in case of the Red group will be assumed to be filled with 0.5 values.  


In [19]:
display(fairness_check({'Blue': blue, 'Red': red}))

,Blue,Red
Demographic Parity,0.50,0.65
Equal Opportunity,0.50,0.75
Predictive Equality,0.50,0.25
Positive Predictive Parity,0.50,0.92
Negative Predictive Parity,0.50,0.57


Results suggest that given decision rule is biased in favour of the priviliaged Blue group.  

Decision rule can be improved by changing rule for the Red group to something more based on statistics and less on random chance.  

## Task 2

I've selected adult income dataset.  

### 2.  
I trained XGBoost and focused fairness evaluation on race attribute.  

In [20]:
display(evaluate_model_fairness(xgboost_model))

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.21,0.24,0.09,0.07,0.10
Equal Opportunity,0.65,0.69,0.57,0.38,0.50
Predictive Equality,0.06,0.08,0.03,0.02,0.04
Positive Predictive Parity,0.79,0.77,0.76,0.71,0.64
Negative Predictive Parity,0.11,0.11,0.06,0.09,0.07


Looking at demographic parity, equal opportunity and predictive equality, we can see that white people, asians and pacific islanders are treated preferentially. Differences in positive predictive parity and negative predictive parity are less pronounced, but show a similar trend.  

Next, I trained Random Forest model.  

In [21]:
display(evaluate_model_fairness(rfc_model))

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.21,0.23,0.09,0.10,0.07
Equal Opportunity,0.61,0.62,0.51,0.38,0.43
Predictive Equality,0.07,0.08,0.03,0.05,0.02
Positive Predictive Parity,0.74,0.74,0.72,0.56,0.75
Negative Predictive Parity,0.13,0.13,0.07,0.09,0.08


Results are quite similiar with the same trend prounounced: a preference towards white poeple, asians and pacific islanders.  

For the next step, I trained XGBoost model, but this time inbalance mitigation was used.  

In [22]:
display(evaluate_model_fairness(xgboost_mitigated_model))

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.22,0.25,0.09,0.06,0.08
Equal Opportunity,0.65,0.71,0.53,0.31,0.50
Predictive Equality,0.07,0.08,0.03,0.02,0.02
Positive Predictive Parity,0.76,0.77,0.72,0.67,0.78
Negative Predictive Parity,0.11,0.10,0.06,0.10,0.07


Bias mitigation has helped, increasing statistics for less privilaged groups, however change was small and inconsistent: some statistics for underprivilaged groups have dropped.  

Performance analysis of trained models:  

In [26]:
display(evaluate_model_performance(models))

,XGBoost,Random Forest,XGBoost with mitigation,Random Forest with mitigation
Accuracy,0.87,0.85,0.87,0.85
Balanced Accuracy,0.79,0.77,0.79,0.77
F1 score,0.71,0.66,0.70,0.66
RoC AuC score,0.79,0.77,0.79,0.77


XGBoost is consistently the best model for this task, beating Random Forest in all metrics. Bias mitigation did not change the accuracy.  

# Appendix

## Task 1

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_sample_weight

pd.options.display.float_format = "{:,.2f}".format

In [2]:
red = {'TP': 60, 'FP': 5, 'FN': 20, 'TN': 15}
blue = {'TP': 25, 'FP': 25, 'FN': 25, 'TN': 25}

In [3]:
def calc_demographic_parity(X):
    return (X['TP']+X['FP']) / sum(X.values())


def calc_equal_opportunity(X):
    return X['TP'] / (X['TP']+X['FN'])


def calc_predictive_equality(X):
    return X['FP'] / (X['FP']+X['TN'])


def calc_positive_predictive_parity(X):
    return X['TP'] / (X['TP']+X['FP'])



def calc_negative_predictive_parity(X):
    return X['FN'] / (X['FN']+X['TN'])


def fairness_check(groups):
    return pd.DataFrame(
            {
                metric_name: [metric_function(X) for X in groups.values()] \
                for metric_name, metric_function in [
                    ('Demographic Parity', calc_demographic_parity),
                    ('Equal Opportunity', calc_equal_opportunity),
                    ('Predictive Equality', calc_predictive_equality),
                    ('Positive Predictive Parity', calc_positive_predictive_parity),
                    ('Negative Predictive Parity', calc_negative_predictive_parity),
                ]
            },
            index=groups.keys()
        ).transpose()

In [4]:
display(fairness_check({'Blue': blue, 'Red': red}))

,Blue,Red
Demographic Parity,0.50,0.65
Equal Opportunity,0.50,0.75
Predictive Equality,0.50,0.25
Positive Predictive Parity,0.50,0.92
Negative Predictive Parity,0.50,0.57


## Task 2

In [5]:
df = pd.read_csv('adult.csv')

numeric_variables = ['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_variables = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']

one_hot_encoder = OneHotEncoder(drop='first')
X_categorical = one_hot_encoder.fit_transform(df[categorical_variables]).toarray()
X_numeric = df[numeric_variables].to_numpy()
X = np.concatenate((X_numeric, X_categorical), axis=1)
y = (df.income == '>50K').astype(np.float32).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2137)

In [6]:
df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [24]:
def train_model(model_constructor, mitigate_bias: bool = False):
    sample_weights = None
    if mitigate_bias:
        race_weights = {race: len(X)/count for race, count in dict(Counter(df.race)).items()}
        X_train_categorical = X_train[:, len(numeric_variables):]
        values = pd.DataFrame(one_hot_encoder.inverse_transform(X_train_categorical), columns=categorical_variables)
        sample_weights = compute_sample_weight(race_weights, values.race)
    return model_constructor().fit(X_train, y_train, sample_weight=sample_weights)


def evaluate_model_fairness(model):
    y_pred = model.predict(X_test)
    X_test_categorical = X_test[:, len(numeric_variables):]
    values = pd.DataFrame(one_hot_encoder.inverse_transform(X_test_categorical), columns=categorical_variables)
    values['y_true'] = y_test
    values['y_pred'] = y_pred

    groups = {}
    for race in values['race'].unique():
        values_in_race = values[values.race == race]
        (tn, fp), (fn, tp) = confusion_matrix(values_in_race.y_true, values_in_race.y_pred)
        groups[race] = {'TP': tp, 'FP': fp, 'FN': fn, 'TN': tn}
    return fairness_check(groups)


def evaluate_model_performance(models, metrics = {
            'Accuracy': accuracy_score,
            'Balanced Accuracy': balanced_accuracy_score,
            'F1 score': f1_score,
            'RoC AuC score': roc_auc_score,
        },
    ):
    return pd.DataFrame(
        {
            name: [method(y_test, model.predict(X_test)) for method in metrics.values()] \
            for name, model in models.items()
        },
        index = metrics.keys()
    )

In [15]:
xgboost_model = train_model(XGBClassifier)
rfc_model = train_model(RandomForestClassifier)
xgboost_mitigated_model = train_model(XGBClassifier, True)
rfc_mitigated_model = train_model(RandomForestClassifier, True)
models = {
    'XGBoost': xgboost_model,
    'Random Forest': rfc_model,
    'XGBoost with mitigation': xgboost_mitigated_model,
    'Random Forest with mitigation': rfc_mitigated_model
}

In [9]:
evaluate_model_fairness(xgboost_model)

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.21,0.24,0.09,0.07,0.10
Equal Opportunity,0.65,0.69,0.57,0.38,0.50
Predictive Equality,0.06,0.08,0.03,0.02,0.04
Positive Predictive Parity,0.79,0.77,0.76,0.71,0.64
Negative Predictive Parity,0.11,0.11,0.06,0.09,0.07


In [10]:
evaluate_model_fairness(rfc_model)

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.21,0.23,0.09,0.10,0.06
Equal Opportunity,0.62,0.66,0.53,0.38,0.43
Predictive Equality,0.07,0.07,0.03,0.05,0.01
Positive Predictive Parity,0.74,0.77,0.74,0.56,0.86
Negative Predictive Parity,0.12,0.12,0.06,0.09,0.08


In [11]:
evaluate_model_fairness(xgboost_mitigated_model)

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.22,0.25,0.09,0.06,0.08
Equal Opportunity,0.65,0.71,0.53,0.31,0.50
Predictive Equality,0.07,0.08,0.03,0.02,0.02
Positive Predictive Parity,0.76,0.77,0.72,0.67,0.78
Negative Predictive Parity,0.11,0.10,0.06,0.10,0.07


In [12]:
evaluate_model_fairness(rfc_mitigated_model)

,White,Asian-Pac-Islander,Black,Amer-Indian-Eskimo,Other
Demographic Parity,0.21,0.23,0.09,0.05,0.06
Equal Opportunity,0.61,0.65,0.51,0.31,0.43
Predictive Equality,0.07,0.07,0.03,0.01,0.00
Positive Predictive Parity,0.74,0.77,0.72,0.80,1.00
Negative Predictive Parity,0.13,0.12,0.07,0.10,0.08


In [25]:
evaluate_model_performance(models)

,XGBoost,Random Forest,XGBoost with mitigation,Random Forest with mitigation
Accuracy,0.87,0.85,0.87,0.85
Balanced Accuracy,0.79,0.77,0.79,0.77
F1 score,0.71,0.66,0.70,0.66
RoC AuC score,0.79,0.77,0.79,0.77
